# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [22]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [23]:
orders = pd.read_csv("Orders.csv",index_col=False)

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [24]:
customer_group = orders.groupby('CustomerID').agg({'amount_spent': 'sum'})
customer_group

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12348,1797.24
12349,1757.55
12350,334.40
...,...
18280,180.60
18281,80.82
18282,178.05


In [25]:
q = customer_group['amount_spent'].quantile(0.8)
top_customers = customer_group[customer_group['amount_spent'] >= q]

top_customers

,amount_spent
CustomerID,
12346,77183.60
12347,4310.00
12352,2506.04
12356,2811.43
12357,6207.67
...,...
18257,2337.63
18259,2338.60
18260,2643.20


In [26]:
customer_group['customer_type'] = 'Preferred'
customer_group.loc[top_customers.index, 'customer_type'] = 'VIP'

customer_group

,amount_spent,customer_type
CustomerID,,
12346,77183.60,VIP
12347,4310.00,VIP
12348,1797.24,Preferred
12349,1757.55,Preferred
12350,334.40,Preferred
...,...,...
18280,180.60,Preferred
18281,80.82,Preferred
18282,178.05,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [27]:
vip_quantile = 0.95
preferred_quantile = 0.75

customer_quantiles = orders.groupby('CustomerID')['amount_spent'].quantile([vip_quantile, preferred_quantile]).unstack()

def label_customer(row):
    if row['amount_spent'] >= customer_quantiles.loc[row['CustomerID'], vip_quantile]:
        return 'VIP'
    elif row['amount_spent'] >= customer_quantiles.loc[row['CustomerID'], preferred_quantile]:
        return 'Preferred'
    else:
        return 'Regular'
orders['CustomerLabel'] = orders.apply(label_customer, axis=1)

vip_customers_by_country = orders[orders['CustomerLabel'] == 'VIP'].groupby('Country')['CustomerID'].nunique()

most_vip_country = vip_customers_by_country.idxmax()
print("The country with the most VIP customers is:", most_vip_country)

The country with the most VIP customers is: United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [28]:
amount_spent_quantiles = orders['amount_spent'].quantile([0, 0.95, 0.99])
orders['customer_category'] = pd.cut(orders['amount_spent'], bins=[-np.inf, amount_spent_quantiles[0.95], amount_spent_quantiles[0.99], np.inf],
                                     labels=['Regular', 'Preferred', 'VIP'])

grouped = orders.groupby(['Country', 'customer_category'])['CustomerID'].nunique().reset_index()

pivoted = grouped.pivot(index='Country', columns='customer_category', values='CustomerID')

pivoted['VIP+Preferred'] = pivoted['VIP'] + pivoted['Preferred']

result = pivoted.sort_values(by='VIP+Preferred', ascending=False)

print(result.head(10))

customer_category  Regular  Preferred  VIP  VIP+Preferred
Country                                                  
United Kingdom        3812       1466  441           1907
Germany                 94         55   15             70
France                  87         38    8             46
Switzerland             21         20    6             26
Spain                   30         17    5             22
Norway                  10          8    7             15
Finland                 12          9    4             13
Portugal                19         10    3             13
Belgium                 25         12    0             12
Channel Islands          7          6    5             11
